In [1]:
import numpy as np
import xtrack as xt
import ApertureCalculator as ac

# Testing aperture classes

In [2]:
ellipse = xt.LimitEllipse(a=0.01, b=0.02)
rectangle = xt.LimitRect(min_x = -0.01, max_x=0.01, min_y=-0.02, max_y=-0.02)
rectellipse = xt.LimitRectEllipse(a=0.01, b=0.02, max_x=0.02, max_y=0.01)
racetrack = xt.LimitRacetrack(a=0.01, b=0.01, min_x = -0.03, max_x=0.03, min_y = -0.02, max_y=0.02)

In [3]:
ap_ellipse = ac.EllipseAperture(ellipse, 'test_ellipse_ap')
ap_rectangle = ac.RectAperture(rectangle, 'test_rectangle_ap')
ap_rectellipse = ac.RectEllipseAperture(rectellipse, 'test_rectellipse_ap')
ap_racetrack = ac.RacetrackAperture(racetrack, 'test_racetrack_ap')

In [4]:
print(f'Ellipse: x extent = {ap_ellipse.compute_x_extent()}, y extent = {ap_ellipse.compute_y_extent()}')
print(f'Rectangle: x extent = {ap_rectangle.compute_x_extent()}, y extent = {ap_rectangle.compute_y_extent()}')
print(f'RectEllipse: x extent = {ap_rectellipse.compute_x_extent()}, y extent = {ap_rectellipse.compute_y_extent()}')
print(f'Racetrack: x extent = {ap_racetrack.compute_x_extent()}, y extent = {ap_racetrack.compute_y_extent()}')

Ellipse: x extent = (np.float64(-0.01), np.float64(0.01)), y extent = (np.float64(-0.02), np.float64(0.02))
Rectangle: x extent = (np.float64(-0.01), np.float64(0.01)), y extent = (np.float64(-0.02), np.float64(-0.02))
RectEllipse: x extent = (np.float64(-0.01), np.float64(0.01)), y extent = (np.float64(-0.01), np.float64(0.01))
Racetrack: x extent = (np.float64(-0.03), np.float64(0.03)), y extent = (np.float64(-0.03), np.float64(0.03))


# Testing Aperture Calculator class

In [2]:
line = xt.Line.from_json('../injection_thin_approx_ap2.json')

Loading line from dict:   0%|          | 0/31427 [00:00<?, ?it/s]

Done loading line from dict.           


In [3]:
line_ap_calc = ac.ApertureCalculator(line)

In [10]:
x_extent = line_ap_calc.compute_x_extent()
y_extent = line_ap_calc.compute_y_extent()

x_min = x_extent[:,0]
x_max = x_extent[:,1]
y_min = y_extent[:,0]
y_max = y_extent[:,1]

# Comparing with previosu functions

## Extent functions

In [13]:
def get_ellipse_x_extent(el):
    if (np.abs(el._cos_rot_s) > 1 and np.abs(el._sin_rot_s) > 1) or (np.abs(el._sin_rot_s) < 0.00001):
        return -el.a + el.shift_x, el.a + el.shift_x
    
    else:
        t_max = np.arctan(-el.b/el.a * el._sin_rot_s/el._cos_rot_s)
        ext = el.a*np.cos(t_max)*el._cos_rot_s-el.b*np.sin(t_max)*el._sin_rot_s
        
        if ext < 0: 
            print('ERROR: ellipse extent is negatif')
            
        return -np.abs(ext) + el.shift_x, np.abs(ext) + el.shift_x
    
def get_rectangle_x_extent(el):
    if (np.abs(el._cos_rot_s) > 1 and np.abs(el._sin_rot_s) > 1) or (np.abs(el._sin_rot_s) < 0.00001):
        return el.min_x+ el.shift_x, el.max_x + el.shift_x
    else:
        w, h = el.max_x-el.min_x, el.max_y-el.min_y
        corners = np.array([[-w/2, -h/2],
                            [w/2, -h/2],
                            [w/2, h/2],
                            [-w/2, h/2]])
        
        rotation_matrix = np.array([
            [el._cos_rot_s, -el._sin_rot_s],
            [el._sin_rot_s,  el._cos_rot_s]
        ])
        
        rotated_corners = corners @ rotation_matrix.T
        
        min_x_val = np.min(rotated_corners[:,0])
        max_x_val = np.max(rotated_corners[:,0])
        
        return min_x_val + el.shift_x, max_x_val + el.shift_x
    
def get_rectellipse_x_extent(el):
    if (np.abs(el._cos_rot_s) > 1 and np.abs(el._sin_rot_s) > 1) or (np.abs(el._sin_rot_s) < 0.00001):
        ext = np.min([el.a, el.max_x])
        
        if ext < 0: 
            print('ERROR: ellipse extent is negatif')
        
        return -np.abs(ext) + el.shift_x, np.abs(ext) + el.shift_x
    
    else:
        #ellipse part
        t_max = np.arctan(-el.b/el.a * el._sin_rot_s/el._cos_rot_s)
        ext_ellipse = np.abs(el.a*np.cos(t_max)*el._cos_rot_s-el.b*np.sin(t_max)*el._sin_rot_s)
        
        #rectangle part
        corners = np.array([[-el.max_x, -el.max_y],
                            [el.max_x, -el.max_y],
                            [el.max_x, el.max_y],
                            [-el.max_x, el.max_y]])
    
        rotation_matrix = np.array([
            [el._cos_rot_s, -el._sin_rot_s],
            [el._sin_rot_s,  el._cos_rot_s]
        ])
        
        rotated_corners = corners @ rotation_matrix.T
        ext_rect = np.max(rotated_corners[:,0])
        if ext_rect < 0:
            print("ERROR: rectangle extent from rectellipse is negatif")
        
        ext = np.min([ext_ellipse, ext_rect])
        if ext < 0:
            print("ERROR: extent from rectellipse is negatif")
        
        return -np.abs(ext) + el.shift_x, np.abs(ext) + el.shift_x

def get_racetrack_x_extent(el):
    if (np.abs(el._cos_rot_s) > 1 and np.abs(el._sin_rot_s) > 1) or (np.abs(el._sin_rot_s) < 0.00001):
        return get_rectangle_x_extent(el)
    
    else:
        rotation_matrix = np.array([
                [el._cos_rot_s, -el._sin_rot_s],
                [el._sin_rot_s,  el._cos_rot_s]
            ])
        
        ellipse_centers = np.array([[el.min_x+el.a, el.min_y+el.b],
                                    [el.max_x -el.a, el.min_y + el.b],
                                    [el.max_x - el.a, el.max_y - el.b],
                                    [el.min_x + el.a, el.max_y - el.b]])
        
        ellipse_centers_rot = ellipse_centers @ rotation_matrix.T
        ellipses = np.array([xt.LimitEllipse(a= el.a, b=el.b, shift_x = ellipse_centers_rot[i,0], shift_y = ellipse_centers_rot[i,1], _cos_rot_s = el._cos_rot_s, _sin_rot_s = el._sin_rot_s) for i in range(4)])
        
        x_exts = []
        for ellipse in ellipses:
            x_min, x_max = get_ellipse_x_extent(ellipse)
            x_exts.append(np.array([x_min, x_max]))
        
        x_exts = np.array(x_exts)
            
        return np.min(x_exts[:,0]) + el.shift_x, np.max(x_exts[:,1]) + el.shift_x

In [14]:
def get_ellipse_y_extent(el):
    if (np.abs(el._cos_rot_s) > 1 and np.abs(el._sin_rot_s) > 1) or (np.abs(el._sin_rot_s) < 0.00001):
        return -el.b + el.shift_y, el.b + el.shift_y
    
    else:
        t_max = np.arctan(el.b/el.a * el._cos_rot_s/el._sin_rot_s)
        ext = el.a*np.cos(t_max)*el._sin_rot_s + el.b*np.sin(t_max)*el._cos_rot_s
        
        if ext < 0: 
            print('ERROR: ellipse extent is negatif')
            
        return -np.abs(ext) + el.shift_y, np.abs(ext) + el.shift_y
    
def get_rectangle_y_extent(el):
    if (np.abs(el._cos_rot_s) > 1 and np.abs(el._sin_rot_s) > 1) or (np.abs(el._sin_rot_s) < 0.00001):
        return el.min_y+ el.shift_y, el.max_y + el.shift_y
    else:
        w, h = el.max_x-el.min_x, el.max_y-el.min_y
        corners = np.array([[-w/2, -h/2],
                            [w/2, -h/2],
                            [w/2, h/2],
                            [-w/2, h/2]])
        
        rotation_matrix = np.array([
            [el._cos_rot_s, -el._sin_rot_s],
            [el._sin_rot_s,  el._cos_rot_s]
        ])
        
        rotated_corners = corners @ rotation_matrix.T
        
        min_y_val = np.min(rotated_corners[:,1])
        max_y_val = np.max(rotated_corners[:,1])
        
        return min_y_val + el.shift_y, max_y_val + el.shift_y
    
def get_rectellipse_y_extent(el):
    if (np.abs(el._cos_rot_s) > 1 and np.abs(el._sin_rot_s) > 1) or (np.abs(el._sin_rot_s) < 0.00001):
        ext = np.min([el.b, el.max_y])
        
        if ext < 0: 
            print('ERROR: ellipse extent is negatif')
        
        return -np.abs(ext) + el.shift_y, np.abs(ext) + el.shift_y
    
    else:
        #ellipse part
        t_max = np.arctan(el.b/el.a * el._cos_rot_s/el._sin_rot_s)
        ext_ellipse = el.a*np.cos(t_max)*el._sin_rot_s + el.b*np.sin(t_max)*el._cos_rot_s
        
        #rectangle part
        corners = np.array([[-el.max_x, -el.max_y],
                            [el.max_x, -el.max_y],
                            [el.max_x, el.max_y],
                            [-el.max_x, el.max_y]])
        
        rotation_matrix = np.array([
            [el._cos_rot_s, -el._sin_rot_s],
            [el._sin_rot_s,  el._cos_rot_s]
        ])
        
        rotated_corners = corners @ rotation_matrix.T
        ext_rect = np.max(rotated_corners[:,1])
        
        if ext_rect < 0:
            print("ERROR: rectangle extent from rectellipse is negatif")
        
        ext = np.min([ext_ellipse, ext_rect])
        if ext < 0:
            print("ERROR: extent from rectellipse is negatif")
        
        return -np.abs(ext) + el.shift_y, np.abs(ext) + el.shift_y

def get_racetrack_y_extent(el):
    if (np.abs(el._cos_rot_s) > 1 and np.abs(el._sin_rot_s) > 1) or (np.abs(el._sin_rot_s) < 0.00001):
        return get_rectangle_y_extent(el)
    
    else:
        rotation_matrix = np.array([
                [el._cos_rot_s, -el._sin_rot_s],
                [el._sin_rot_s,  el._cos_rot_s]
            ])
        
        ellipse_centers = np.array([[el.min_x+el.a, el.min_y+el.b],
                                    [el.max_x -el.a, el.min_y + el.b],
                                    [el.max_x - el.a, el.max_y - el.b],
                                    [el.min_x + el.a, el.max_y - el.b]])
        
        ellipse_centers_rot = ellipse_centers @ rotation_matrix.T
        ellipses = np.array([xt.LimitEllipse(a= el.a, b=el.b, shift_x = ellipse_centers_rot[i,0], shift_y = ellipse_centers_rot[i,1], _cos_rot_s = el._cos_rot_s, _sin_rot_s = el._sin_rot_s) for i in range(4)])
        
        y_exts = []
        for ellipse in ellipses:
            y_min, y_max = get_ellipse_y_extent(ellipse)
            y_exts.append(np.array([y_min, y_max]))
        
        y_exts = np.array(y_exts)
            
        return np.min(y_exts[:,0]) + el.shift_y, np.max(y_exts[:,1]) + el.shift_y

In [15]:
def get_x_extent(el):
    if el.__class__.__name__ == 'LimitRect':
        return get_rectangle_x_extent(el)
    
    elif el.__class__.__name__ == 'LimitEllipse':
        return get_ellipse_x_extent(el)
    
    elif el.__class__.__name__ == 'LimitRectEllipse':
        return get_rectellipse_x_extent(el)
    
    elif el.__class__.__name__ == 'LimitRacetrack':
        return get_racetrack_x_extent(el)

def get_y_extent(el):
    if el.__class__.__name__ == 'LimitRect':
        return get_rectangle_y_extent(el)
    
    elif el.__class__.__name__ == 'LimitEllipse':
        return get_ellipse_y_extent(el)
    
    elif el.__class__.__name__ == 'LimitRectEllipse':
        return get_rectellipse_y_extent(el)
    
    elif el.__class__.__name__ == 'LimitRacetrack':
        return get_racetrack_y_extent(el)

## Making x and y extents from functions

In [6]:
tab = line.get_table()
mask = [el.startswith('Limit') for el in tab.element_type]

In [16]:
x_max_func = np.array([get_x_extent(line[name])[1] for name in tab.name[mask]])
x_min_func = np.array([get_x_extent(line[name])[0] for name in tab.name[mask]])

y_max_func = np.array([get_y_extent(line[name])[1] for name in tab.name[mask]])
y_min_func = np.array([get_y_extent(line[name])[0] for name in tab.name[mask]])

names_arr = np.array(tab.name[mask])
s_arr = np.array(tab.s[mask])

## Comparing

In [21]:
print(np.unique(np.subtract(x_max, x_max_func)))
print(np.unique(np.subtract(x_min, x_min_func)))
print(np.unique(np.subtract(y_max, y_max_func)))
print(np.unique(np.subtract(y_min, y_min_func)))

[0.]
[0.]
[0.]
[0.]
